In [172]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd
import os
import dotenv
import json
import re

dotenv.load_dotenv(override=True)

False

In [214]:
tasks = ['date_understanding', 'geometric_shapes', 'navigate', 'temporal_sequences', 'tracking_shuffled_objects_three_objects',
        'tracking_shuffled_objects_seven_objects', 'object_counting', 'penguins_in_a_table', 'reasoning_about_colored_objects']

task = tasks[8]

OUTPUT_DIR = f"../outputs/{task}"

In [215]:
models = os.listdir(OUTPUT_DIR)

In [216]:
models.remove('.ipynb_checkpoints')

ValueError: list.remove(x): x not in list

In [217]:
models.sort()
models

['with_image', 'without_image']

In [218]:
# Upload base dataset
df = pd.read_json(os.path.join(OUTPUT_DIR, models[0], "output.jsonl"), lines=True)
woi_df = pd.read_json(os.path.join(OUTPUT_DIR, models[1], "output.jsonl"), lines=True)

df['image'] = df.apply(lambda x: f'https://automated-vqa.s3.amazonaws.com/{task}/'+str(x['qid'])+'.jpg', axis=1)
base_df = pd.DataFrame({
    "qid": df["qid"],
    "image": df['image'],
    "question": df["question"],
    "answer": df["answer"].astype(str) ,
    "with_image_correct": df["is_correct"].astype(bool),
    "without_image_correct": woi_df["is_correct"].astype(bool)
})

In [219]:
zeno_client = ZenoClient("zen_P4CK880bWHV2dJLbAenb0r8Gf6QNLdTXhSkkHqPDm4I")

/data/tir/projects/tir4/users/sakter/anaconda3/envs/bleed/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package zeno-client is out of date. Your version is 0.1.13, the latest is 0.1.14.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [220]:
base_df["data"] = base_df.apply(
    lambda row: {"question": row.question, "image": row.image}, axis=1
)

project = zeno_client.create_project(
    name=f"VLM: {task.upper()}",
    description=f"Evaluation of VLM on {task.upper()} dataset",
    view={
        "data": {
            "type": "vstack",
            "keys": {"question": {"type": "text"}, "image": {"type": "image"}},
        },
        "label": {
            "type": "text"
        },
        "output": {
            "type": "markdown"
        }
    },
    public=True,
    metrics=[
        ZenoMetric(name="Accuracy Strict Match", type="mean", columns=["is_correct"]),
        ZenoMetric(name="Accuracy", type="mean", columns=["is_correct_last"])
    ],
)

Successfully created project.
Access your project at  https://hub.zenoml.com/project/13c4f17f-aa27-4205-b760-14b92a1df7a4/VLM%3A%20REASONING_ABOUT_COLORED_OBJECTS


In [221]:
project.upload_dataset(base_df, id_column="qid", data_column="data", label_column="answer")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded data


In [222]:
def check_fraction(num):
    if num % 1 != 0:
        return 1
    return 0

for model in models:
    df = pd.read_json(os.path.join(OUTPUT_DIR, model, "output.jsonl"), lines=True)
    if model == 'with_image':
        df['image'] = df.apply(lambda x: 'https://automated-vqa.s3.amazonaws.com/mawpsmultiarith/dummy_img.png', axis=1)
    if model == 'mixtral':
        output_df = pd.DataFrame({
            "qid": df["qid"],
            "output": df.apply(lambda x: f"{x['response'].split('Q:')[0]}\n\n**{x['predict']}**", axis=1),
            "question_length": df.apply(lambda x: len(x['question'].split('Q:')[1].strip().split(' ')), axis=1),
            "output_length": df.apply(lambda x: len(x['response'].split('Q:')[0].split(' ')), axis=1),
            "is_correct": df["is_correct"].astype(bool),
            "is_correct_last": df["is_correct_last"].astype(bool),
        })
    else:
        output_df = pd.DataFrame({
            "qid": df["qid"],
            "output": df.apply(lambda x: f"{x['response'].split('Q:')[0]}\n\n**{x['predict']}**", axis=1),
            "question_length": df.apply(lambda x: len(x['question'].split('Q:')[1].strip().split(' ')), axis=1),
            "output_length": df.apply(lambda x: len(x['response'].split(' ')), axis=1),
            "is_correct": df["is_correct"].astype(bool),
            "is_correct_last": df["is_correct_last"].astype(bool)
        })
    if model == 'gpt-4-1106-preview':
        model = 'gpt-4-turbo'
    project.upload_system(output_df, name=model, id_column="qid", output_column="output")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system
